In [152]:
import re
import pandas as pd

In [4]:
import httplib2

from apiclient.discovery import build

from oauth2client.client import flow_from_clientsecrets
from oauth2client.file import Storage
from oauth2client import tools

# Enter your Google Developer Project number
PROJECT_NUMBER = '500432946938'

FLOW = flow_from_clientsecrets('../client_secret.json',
                               scope='https://www.googleapis.com/auth/bigquery')

storage = Storage('bigquery_credentials.dat')
credentials = storage.get()

if credentials is None or credentials.invalid:
  # Run oauth2 flow with default arguments.
  credentials = tools.run_flow(FLOW, storage, tools.argparser.parse_args([]))

http = httplib2.Http()
http = credentials.authorize(http)

bigquery_service = build('bigquery', 'v2', http=http)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&response_type=code&client_id=500432946938-2id4fr1k894lrhjr2prqe2q2tivp6630.apps.googleusercontent.com&access_type=offline

If your browser is on a different machine then exit and re-run this
application with the command-line parameter 

  --noauth_local_webserver

Authentication successful.


In [163]:
YOUTUBE_REGEX = '(?:https?:\/\/)?(?:youtu\.be\/|(?:www\.)?youtube\.com\/watch(?:\.php)?\?.*v=)([a-zA-Z0-9\-_]+)'

def get_link_data(link_id):
    youtube_link_ids = set()
    ncomments = 0

    for i in range(1, 7):
        query = {'query':'SELECT body FROM [fh-bigquery:reddit_comments.2015_{:02}] WHERE '
            'link_id="{}"'.format(i, link_id)
        }
        query_request = bigquery_service.jobs()
        query_response = query_request.query(projectId=PROJECT_NUMBER,
                                             body=query).execute()
        ncomments += int(query_response['totalRows'])

        try:
            rows = (r['f'][0]['v'] for r in query_response['rows'])
            links = (re.findall(YOUTUBE_REGEX, r) for r in rows)
            youtube_link_ids.update(sum(list(links), []))
        except KeyError:
            continue

    return ncomments, youtube_link_ids

In [95]:
get_link_data(link_id)

{'query': 'SELECT body FROM [fh-bigquery:reddit_comments.2015_01] WHERE link_id="t3_39bn9p"'}
{'query': 'SELECT body FROM [fh-bigquery:reddit_comments.2015_02] WHERE link_id="t3_39bn9p"'}
{'query': 'SELECT body FROM [fh-bigquery:reddit_comments.2015_03] WHERE link_id="t3_39bn9p"'}
{'query': 'SELECT body FROM [fh-bigquery:reddit_comments.2015_04] WHERE link_id="t3_39bn9p"'}
{'query': 'SELECT body FROM [fh-bigquery:reddit_comments.2015_05] WHERE link_id="t3_39bn9p"'}
{'query': 'SELECT body FROM [fh-bigquery:reddit_comments.2015_06] WHERE link_id="t3_39bn9p"'}


(65,
 {u'DduNRXh3h3o',
  u'IK5I4cTkL-E',
  u'Iimujttqr_A',
  u'Z6ih1aKeETk',
  u'bmatItacEvM',
  u'clnozSXyF4k',
  u'gpGxU1tPu3g',
  u'kqJ2iZ8HJcc',
  u'uSIbpqNB5Fo',
  u'zqLkmTWRt0g'})

In [110]:
def get_all_submission_link_id():
    link_ids = set()

    for i in range(1, 7):
        query = {'query': 'SELECT link_id FROM [fh-bigquery:reddit_comments.2015_{:02}] WHERE '
            'subreddit="listentothis" GROUP BY link_id'.format(i)
        }
        
        query_request = bigquery_service.jobs()
        query_response = query_request.query(projectId=PROJECT_NUMBER,
                                             body=query).execute()
        try:
            rows = (r['f'][0]['v'] for r in query_response['rows'])
            link_ids.update(rows)
        except KeyError:
            continue

    return link_ids

In [111]:
link_ids = get_all_submission_link_id()

In [112]:
len(link_ids)

33984

In [153]:
df = pd.DataFrame()

In [155]:
df['link_id'] = list(link_ids)

In [114]:
import praw

In [199]:
USERNAME = 'arXiv_landing_bot'
PASSWORD = 'KIPHJOOqE1Gc'

r = praw.Reddit(user_agent='arXiv_landing')
r.login(USERNAME, PASSWORD)


In [231]:
def chunks(iterable, chunk_size):
    """Generate sequences of `chunk_size` elements from `iterable`."""
    iterable = iter(iterable)
    while True:
        chunk = []
        try:
            for _ in range(chunk_size):
                chunk.append(iterable.next())
            yield chunk
        except StopIteration:
            if chunk:
                yield chunk
            break

In [232]:
df['upvotes'] = 0
df['youtube_id'] = ''

i = 0
for chunk in chunks(list(df['link_id']), 100):
    print(i)
    submissions = r.get_info(thing_id=chunk)
    if submissions is None:
        i += len(chunk)
        continue
    
    for s in submissions:
        youtube_id = re.findall(YOUTUBE_REGEX, s.url)
        youtube_id = youtube_id[0] if youtube_id else ''
        df['youtube_id'].ix[i] = youtube_id
        df['upvotes'].ix[i] = s.ups
        i += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [274]:
df2 = df[df.youtube_id != '']
df2 = df2.sort('upvotes', ascending=False).reset_index()

In [249]:
import pafy

In [261]:
url = "https://www.youtube.com/watch?v=Sw9uicEGjGw"

In [262]:
video = pafy.new(url)

In [263]:
video.title

u'Kung Fury Full Soundtrack'

In [275]:
df2['title'] = ''

In [283]:
for i, youtube_id in enumerate(df2['youtube_id']):
    try:
        url = "https://www.youtube.com/watch?v={}".format(youtube_id)
        video = pafy.new(url)
        df2['title'].ix[i] = video.title
        print(i)
    except (KeyError, IOError, AttributeError):
        continue


0
1
2
3
4
5
6
7
8
9
10
11
12
13
15
16
17
18
19
20
21
22
24
25
26
27
28
29
30
31
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
106
107
108
109
110
111
112
113
114
115
116
117
118
120
121
123
124
125
126
127
128
129
130
132
133
134
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
157
158
159
160
161
162
163
164
165
166
167
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
2

AttributeError: 'NoneType' object has no attribute 'groups'

In [286]:
df2.save('data2.df')

/home/eben/projects/listentothis/venv/local/lib/python2.7/site-packages/pandas/core/generic.py:1000: FutureWarning: save is deprecated, use to_pickle
  warnings.warn("save is deprecated, use to_pickle", FutureWarning)


In [291]:
df2

,index,link_id,upvotes,youtube_id,title
0,21263,t3_33h11f,3451,CXPADwU05OQ,[MV] Lim Kim(김예림) (Togeworl(투개월)) _ Awoo
1,11730,t3_35jvwu,3125,b0eFx5a-FMg,"The Moth & The Flame ""Young & Unafraid"""
2,10510,t3_361t1p,2416,37eEUsd1ASA,DATA - Don't Sing feat. Benny Sings (official ...
3,8103,t3_315cjb,2121,ewOzi5-AZXU,"B-Real, Coolio, Method Man, LL Cool J And Bust..."
4,14229,t3_2sy3cx,2039,JCHTX_pgw6A,BADBADNOTGOOD - TITLE THEME / SARIA'S SONG / S...
5,19395,t3_2en7r4,1923,jRHQPG1xd9o,VULFPECK /// 1612
6,3671,t3_2rah9u,1910,gtg-mK5E7Zw,HINTO 『かなしみアップデイト』
7,27430,t3_2gsd09,1893,FRSr0GprIIw,The Trouble | Pogo & Jeesh
8,848,t3_2hrgtb,1841,r4G0nbpLySI,VULFPECK /// Wait for the Moment
9,26658,t3_2gowpx,1751,ocGiulPm3IU,Lil Dicky - Lemme Freak (Official Video)
